In [1]:
import numpy as np
import pandas as pd
from pulp import *

In [2]:
demand = pd.read_excel("demand.xlsx")
demand.set_index("Shop",inplace=True)

inboundcost = pd.read_excel("plant+to+warehouse.xlsx")
inboundcost=inboundcost.rename(columns={"Unnamed: 0":"Plant"})
inboundcost.set_index("Plant",inplace=True)

outboundcost = pd.read_excel("outbound_distance.xlsx")
outboundcost.set_index("warehouse",inplace=True)

prodtable = pd.read_excel("plant.xlsx")
prodtable.set_index("Unnamed: 0",inplace=True)

outboundhandling = pd.read_excel("wrehouse_costs.xlsx")
outboundhandling.set_index("warehouse",inplace=True)

In [3]:
# building variables for LP model
plantusekey = prodtable.index.to_list()
plantvar = LpVariable.dicts("plantusekey",plantusekey,cat="Binary")

warehouseusekey = outboundhandling.index.to_list()
warehousevar = LpVariable.dicts("warehousekey",warehouseusekey,cat="Binary")

# list of shops
shopkey = demand.index.to_list()
# list of products
prodkey = demand.columns.to_list()

# list of plant-warehouses for inbound movement
plant_wh_key = [(p,w,r) for p in plantusekey for w in warehouseusekey for r in prodkey]
plant_wh_var = LpVariable.dicts("plant_wh",plant_wh_key,lowBound=0,upBound=None,cat="Integer")

# list of wh-shops for outbound movement
wh_shop_key = [(w,s,r) for w in warehouseusekey for s in shopkey for r in prodkey]
wh_shop_var = LpVariable.dicts("wh_shop",wh_shop_key,lowBound=0,upBound=None,cat="Integer")


In [4]:
# Objective function

totinboundcost = lpSum(plant_wh_var[(p,w,r)]*inboundcost.loc[p,w] for p in plantusekey for w in warehouseusekey for r in prodkey)
totoutboundcost = lpSum(wh_shop_var[(w,s,r)]*outboundcost.loc[w,s] for w in warehouseusekey for s in shopkey for r in prodkey)

openplantcost = lpSum(plantvar[p]*prodtable.loc[p,"Fixed Cost"] for p in plantusekey)
productioncost = lpSum(plant_wh_var[(p,w,r)]*prodtable.loc[p,r] for p in plantusekey for w in warehouseusekey for r in prodkey)

opendccost = lpSum(warehousevar[w]*outboundhandling.loc[w,"Fixed Cost"] for w in warehouseusekey)
warehousingcost = lpSum(wh_shop_var[(w,s,r)]*outboundhandling.loc[w,r] for w in warehouseusekey for s in shopkey for r in prodkey)

In [5]:
model = LpProblem("model",sense=LpMinimize)
model += totinboundcost+totoutboundcost+openplantcost+productioncost+opendccost+warehousingcost

In [6]:
# production plant constraints
for p in plantusekey:
    model += lpSum(plant_wh_var[(p,w,r)] for w in warehouseusekey for r in prodkey) <= prodtable.loc[p,"Capacity"]*plantvar[p]

# warehousing constraints
for w in warehouseusekey:
    model += lpSum(wh_shop_var[(w,s,r)] for s in shopkey for r in prodkey) <= outboundhandling.loc[w,"Capacity"]*warehousevar[w]

# demand constraints
for s in shopkey:
    for r in prodkey:
        model += lpSum(wh_shop_var[(w,s,r)] for w in warehouseusekey) >= demand.loc[s,r]

# flow constraints
for w in warehouseusekey:
    for r in prodkey:
        model += lpSum(wh_shop_var[(w,s,r)] for s in shopkey) == lpSum(plant_wh_var[(p,w,r)] for p in plantusekey)

In [7]:
model.solve()

1

In [8]:
value(model.objective)

8813636.2

In [9]:
soldict = {}
for z in model.variables():
    if z.varValue >0:
        print (z.name, "=" ,z.varValue)
    else:
        continue

plant_wh_('Plant_1',_'warehouse_4',_'tables') = 844
plant_wh_('Plant_1',_'warehouse_6',_'Chairs') = 1109
plant_wh_('Plant_1',_'warehouse_6',_'tables') = 1877
plant_wh_('Plant_3',_'warehouse_1',_'Curtains') = 800
plant_wh_('Plant_3',_'warehouse_4',_'Beds') = 765
plant_wh_('Plant_3',_'warehouse_4',_'Chairs') = 555
plant_wh_('Plant_3',_'warehouse_4',_'Curtains') = 866
plant_wh_('Plant_3',_'warehouse_6',_'Beds') = 921
plant_wh_('Plant_3',_'warehouse_6',_'Curtains') = 1093
plant_wh_('plant_2',_'warehouse_1',_'Beds') = 1149
plant_wh_('plant_2',_'warehouse_1',_'Chairs') = 943
plant_wh_('plant_2',_'warehouse_4',_'Chairs') = 190
plantusekey_Plant_1 = 1
plantusekey_Plant_3 = 1
plantusekey_plant_2 = 1
warehousekey_warehouse_1 = 1
warehousekey_warehouse_4 = 1
warehousekey_warehouse_6 = 1
wh_shop_('warehouse_1',_'shop_10',_'Beds') = 155
wh_shop_('warehouse_1',_'shop_10',_'Chairs') = 189
wh_shop_('warehouse_1',_'shop_10',_'Curtains') = 223
wh_shop_('warehouse_1',_'shop_11',_'Beds') = 150
wh_shop_('w

In [10]:
soldict = {}
for z in model.variables():
    if z.varValue >0:
        soldict[z.name] = z.varValue
    else:
        continue

In [11]:
soldict

{"plant_wh_('Plant_1',_'warehouse_4',_'tables')": 844,
 "plant_wh_('Plant_1',_'warehouse_6',_'Chairs')": 1109,
 "plant_wh_('Plant_1',_'warehouse_6',_'tables')": 1877,
 "plant_wh_('Plant_3',_'warehouse_1',_'Curtains')": 800,
 "plant_wh_('Plant_3',_'warehouse_4',_'Beds')": 765,
 "plant_wh_('Plant_3',_'warehouse_4',_'Chairs')": 555,
 "plant_wh_('Plant_3',_'warehouse_4',_'Curtains')": 866,
 "plant_wh_('Plant_3',_'warehouse_6',_'Beds')": 921,
 "plant_wh_('Plant_3',_'warehouse_6',_'Curtains')": 1093,
 "plant_wh_('plant_2',_'warehouse_1',_'Beds')": 1149,
 "plant_wh_('plant_2',_'warehouse_1',_'Chairs')": 943,
 "plant_wh_('plant_2',_'warehouse_4',_'Chairs')": 190,
 'plantusekey_Plant_1': 1,
 'plantusekey_Plant_3': 1,
 'plantusekey_plant_2': 1,
 'warehousekey_warehouse_1': 1,
 'warehousekey_warehouse_4': 1,
 'warehousekey_warehouse_6': 1,
 "wh_shop_('warehouse_1',_'shop_10',_'Beds')": 155,
 "wh_shop_('warehouse_1',_'shop_10',_'Chairs')": 189,
 "wh_shop_('warehouse_1',_'shop_10',_'Curtains')": 22

In [12]:
df = pd.DataFrame(list(soldict.items()),columns=['Variable Name','Variable Value'])
df

,Variable Name,Variable Value
0,"plant_wh_('Plant_1',_'warehouse_4',_'tables')",844
1,"plant_wh_('Plant_1',_'warehouse_6',_'Chairs')",1109
2,"plant_wh_('Plant_1',_'warehouse_6',_'tables')",1877
3,"plant_wh_('Plant_3',_'warehouse_1',_'Curtains')",800
4,"plant_wh_('Plant_3',_'warehouse_4',_'Beds')",765
...,...,...
86,"wh_shop_('warehouse_6',_'shop_6',_'Curtains')",165
87,"wh_shop_('warehouse_6',_'shop_6',_'tables')",160
88,"wh_shop_('warehouse_6',_'shop_7',_'tables')",200
89,"wh_shop_('warehouse_6',_'shop_8',_'tables')",105


In [14]:
dist_matrix = {(w,s):1 if outboundcost.loc[w,s] <=170 else 0 for w in warehouseusekey for s in shopkey}

disttravel = lpSum(wh_shop_var[w,s,r].varValue*dist_matrix[(w,s)] for w in warehouseusekey for s in shopkey for r in prodkey)

disttravel

8718

In [16]:
totdemand = lpSum(demand.loc[s,r] for s in shopkey for r in prodkey)
totdemand

11112

In [18]:
servicelevel = disttravel/totdemand
servicelevel

0.7845572354211663